In [2]:
import pandas as pd
import numpy as np

In [3]:
pm10_2004 = pd.read_csv("data/mappm102004g.csv")

pm10_2004.head()

/tmp/ipykernel_5340/310841554.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  pm10_2004 = pd.read_csv("data/mappm102004g.csv")


,pm10,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,2004,NaN,NaN,NaN
1,annual mean,NaN,NaN,NaN
2,ug m-3 (gravimetric units),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,ukgridcode,x,y,pm102004g


In [4]:
pm10_2004.drop(labels=[0,1,2,3,4], inplace=True)

pm10_2004.rename(columns={"pm10": "ukgridcode", "Unnamed: 1": "x", "Unnamed: 2": "y", "Unnamed: 3": "pm10"}, inplace=True)

pm10_2004.head()

,ukgridcode,x,y,pm10
5,54291,460500,1221500,MISSING
6,54292,461500,1221500,MISSING
7,54294,463500,1221500,MISSING
8,54979,458500,1220500,MISSING
9,54980,459500,1220500,MISSING


In [5]:
pm10_2004["pm10"] = pd.to_numeric(pm10_2004["pm10"], errors="coerce")

In [6]:
pm10_2004 = pm10_2004.astype({"x": "int", "y": "int"})

In [7]:
x_coords = np.sort(pm10_2004["x"].unique())
y_coords = np.sort(pm10_2004["y"].unique())

In [8]:
raster = pm10_2004.pivot(index="y", columns="x", values="pm10").sort_index(ascending=False)


In [9]:
import rasterio
from rasterio.transform import from_origin

In [ ]:
pixel_size = 1000
transform = from_origin(x_coords.min(), y_coords.max(), pixel_size, pixel_size)

with rasterio.open(
    "data/pollution_pm10_2004.tif",
    "w",
    driver="GTiff",
    height=raster.shape[0],
    width=raster.shape[1],
    count=1,
    dtype="float32",
    crs="EPSG:27700",
    transform=transform,
    nodata=np.nan,
) as dst:
    dst.write(raster.values.astype("float32"), 1)

In [12]:
from rasterstats import zonal_stats
import geopandas as gpd

In [ ]:
lsoa = gpd.read_file("data/infuse_msoa_lyr_2011_clipped.shp").to_crs("EPSG:27700")

In [ ]:
batch_size = 1000
results = []

for start in range(0, len(lsoa), batch_size):
    end = start + batch_size
    subset = lsoa.iloc[start:end]
    print(f"Processing {start}–{end} / {len(lsoa)}")

    stats = zonal_stats(
        subset,
        "data/pollution_pm10_2004.tif",
        stats=["mean"],
        nodata=np.nan,
        all_touched=True,
        geojson_out=False
    )
    # attach LSOA codes
    df = pd.DataFrame(stats)
    df["LSOA11CD"] = subset["LSOA11CD"].values
    results.append(df)

Processing 0–1000 / 34753
Processing 1000–2000 / 34753
Processing 2000–3000 / 34753
Processing 3000–4000 / 34753
Processing 4000–5000 / 34753
Processing 5000–6000 / 34753
Processing 6000–7000 / 34753
Processing 7000–8000 / 34753
Processing 8000–9000 / 34753
Processing 9000–10000 / 34753
Processing 10000–11000 / 34753
Processing 11000–12000 / 34753
Processing 12000–13000 / 34753
Processing 13000–14000 / 34753
Processing 14000–15000 / 34753
Processing 15000–16000 / 34753
Processing 16000–17000 / 34753
Processing 17000–18000 / 34753
Processing 18000–19000 / 34753
Processing 19000–20000 / 34753
Processing 20000–21000 / 34753
Processing 21000–22000 / 34753
Processing 22000–23000 / 34753
Processing 23000–24000 / 34753
Processing 24000–25000 / 34753
Processing 25000–26000 / 34753
Processing 26000–27000 / 34753
Processing 27000–28000 / 34753
Processing 28000–29000 / 34753
Processing 29000–30000 / 34753
Processing 30000–31000 / 34753
Processing 31000–32000 / 34753
Processing 32000–33000 / 34753

In [15]:
lsoa_pm10 = pd.concat(results, ignore_index=True)

In [21]:
lsoa_pm10.rename(columns={"mean": "pm10_2004"}, inplace=True)
lsoa_pm10.head()

,pm10_2004,LSOA11CD
0,22.450075,E01000001
1,22.500353,E01000002
2,22.808451,E01000003
3,22.217785,E01000005
4,22.865450,E01000006
